# Advanced Machine Learning: Progetto

## Parte 1 - Data Processing

Progetto finale del corso di Advanced Machine Learning
<br>Anno Accademico: 2018/2019
<br>Studenti:
- Mattia Pennati - 793375
- Mirko Rima - 793435
- Francesco Rovera - 794194

## 1) Import

###1.1) Import librerie

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy import stats
import matplotlib.pyplot as plt
import datetime
import holidays
import geopy.distance
from google.colab import drive
from google.colab import files
import re
import math
from math import radians, cos, sin, asin, sqrt

### 1.2) Import dei Dataset
Connettere COLAB e Google Drive e importare i dataset

In [0]:
drive.mount('/content/drive')
train = np.genfromtxt('drive/My Drive/AML_TaxiTripDuration_Pennati_Rima_Rovera/Dataset/train/train.csv', delimiter=',', dtype='unicode')
test = np.genfromtxt('drive/My Drive/AML_TaxiTripDuration_Pennati_Rima_Rovera/Dataset/test/test.csv', delimiter=',', dtype='unicode')
weather = np.genfromtxt('drive/My Drive/AML_TaxiTripDuration_Pennati_Rima_Rovera/Dataset/weather/Weather.csv', delimiter=',', dtype='unicode')
zone = np.genfromtxt('drive/My Drive/AML_TaxiTripDuration_Pennati_Rima_Rovera/Dataset/NY_borough/NY_borough.csv', delimiter=',', dtype='unicode')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##2) Data processing


###2.1) Modifiche pre-join ai dataset

Eliminare le colonne considerate inutili ai fini del machine learning dai diversi dataset, dividere la data in data e ora e convertire i campi testuali in numerici

In [0]:
# Eliminare le colonne indesiderate (meteo: 27,28,29 condizioni mai registrate)
weather = np.delete(weather, [2,4,7,9,10,11,13,15,17,19,21,23,27,28,29], axis=1)
train = np.delete(train, [0,1,3,9], axis=1)
test = np.delete(test, [0,1,8], axis=1)

# Inserire le colonne 'date' e 'hour', rimuovere la precedente colonna 'date'
# e mantenere solo l'ora (ignorare minuti e secondi)
weather = np.insert(weather, 0, 'date', axis=1)
weather = np.insert(weather, 1, 'hour', axis=1)
for i in range(1,weather.shape[0]):
  date = weather[i,2].split(' ')
  weather[i,0]=date[0]
  weather[i,1]=date[1].split(':')[0]
weather = np.delete(weather, [2], axis=1)
train = np.insert(train, 0, 'date', axis=1)
train = np.insert(train, 1, 'hour', axis=1)
for i in range(1,train.shape[0]):
  date = train[i,2].split(' ')
  train[i,0]=date[0]
  train[i,1]=date[1].split(':')[0]
train = np.delete(train, [2], axis=1)
test = np.insert(test, 0, 'date', axis=1)
test = np.insert(test, 1, 'hour', axis=1)
for i in range(1,test.shape[0]):
  date = test[i,2].split(' ')
  test[i,0]=date[0]
  test[i,1]=date[1].split(':')[0]
test = np.delete(test, [2], axis=1)

# Rimuovere le voci del 2015 e mantenere un solo valore per ora
toDelete = []
for i in range(1,weather.shape[0]):
  if(weather[i,0].find("2015") != -1):
    toDelete = np.append(toDelete,i)
weather = np.delete(weather,[toDelete], axis=0)
for i in range(2,weather.shape[0]):
  previousdate = weather[i-1,0]
  previoushour = weather[i-1,1]
  if((weather[i,0]==previousdate) and (weather[i,1]==previoushour)):
    toDelete = np.append(toDelete,i)
weather = np.delete(weather, [toDelete], axis=0)

# La colonna 'cond' da String a Integer
for i in range (1,weather.shape[0]):
  a = weather[i,12]
  if(a == "Clear"):
    weather[i,12] = 0
  if(a == "Overcast"):
    weather[i,12] = 0
  if(a == "Mostly Cloudy"):
    weather[i,12] = 0
  if(a == "Partly Cloudy"):
    weather[i,12] = 0
  if(a == "Scattered Clouds"):
    weather[i,12] = 0
  if(a == "Light Rain"):
    weather[i,12] = 1
  if(a == "Unknown"):
    weather[i,12] = 6
  if(a == "Haze"):
    weather[i,12] = 2
  if(a == "Light Snow"):
    weather[i,12] = 3
  if(a == "Rain"):
    weather[i,12] = 1
  if(a == "Heavy Rain"):
    weather[i,12] = 4
  if(a == "Snow"):
    weather[i,12] = 3
  if(a == "Light Freezing Rain"):
    weather[i,12] = 3
  if(a == "Heavy Snow"):
    weather[i,12] = 5

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


### 2.2) Join  dei datasets
fra i dataset dei taxi con quello delle condizioni metereologiche con chiave <Data,Ora>

In [0]:
# Convertire il campo data in formato data
for i in range(1,weather.shape[0]):
  weather[i][0] = pd.to_datetime(weather[i][0],format='%Y-%m-%d')
for i in range(1,train.shape[0]):
  train[i][0] = pd.to_datetime(train[i][0], format='%Y-%m-%d')
for i in range(1,test.shape[0]):
  test[i][0] = pd.to_datetime(test[i][0], format='%Y-%m-%d')

# Join
weather = pd.DataFrame(weather)
train = pd.DataFrame(train)
test = pd.DataFrame(test)
train = pd.merge(train, weather, how='inner', on=[0,1])
train = train.values
test = pd.merge(test, weather, how='inner', on=[0,1])
test = test.values

###2.3) Modifiche post-join 

Cambiare il formato della data, rimuoverla come colonna e inserire le colonne mese, giorno della settimana e giorno lavorativo/festivo (tramite)

In [0]:
# Cambiare il formato della data
for i in range(1,train.shape[0]):
  train[i,0] = train[i,0].split(' ')[0]
for i in range(1,train.shape[0]):
  train[i,0] = train[i,0].split(' ')[0]
for i in range(1,test.shape[0]):
  test[i,0] = test[i,0].split(' ')[0]
for i in range(1,test.shape[0]):
  test[i,0] = test[i,0].split(' ')[0]

# Ottenere i giorni festivi e creare le 3 nuove colonne
us_holidays = holidays.UnitedStates()
#Train
train = np.insert(train, train.shape[1], 'dayweek', axis=1)
train = np.insert(train, train.shape[1], 'holidays', axis=1)
train = np.insert(train, train.shape[1], 'month', axis=1)
for i in range(1,train.shape[0]):
    d = train[i][0]
    y,m,d = d.split("-")
    data = datetime.date((int)(y), (int)(m), (int)(d))
    train[i][train.shape[1]-3] = data.weekday()
    if (train[i][train.shape[1]-3] == 6):
      train[i][train.shape[1]-2] = 1
    else:
      if(train[i][train.shape[1]-2] == True ):
        train[i][train.shape[1]-2] = 1
      else:     
        train[i][train.shape[1]-2] = 0
    train[i,train.shape[1]-1] = (int)(m)
train = np.delete(train, [0], axis=1)
#Test
test = np.insert(test, test.shape[1], 'dayweek', axis=1)
test = np.insert(test, test.shape[1], 'holidays', axis=1)
test = np.insert(test, test.shape[1], 'month', axis=1)
for i in range(1,test.shape[0]):
    d = test[i][0]
    y,m,d = d.split("-")
    data = datetime.date((int)(y), (int)(m), (int)(d))
    test[i][test.shape[1]-3] = data.weekday()
    if (test[i][test.shape[1]-3] == 6):
      test[i][test.shape[1]-2] = 1
    else:
      if(test[i][test.shape[1]-2] == True ):
        test[i][test.shape[1]-2] = 1
      else:     
        test[i][test.shape[1]-2] = 0
    test[i,test.shape[1]-1] = (int)(m)
test = np.delete(test, [0], axis=1)

Aggiungere una colonna 'distance' calcolandola dalle coordinate GPS (ha la pecca di essere una distanza in linea d'aria) ed una 'speed' (distance/trip duration)

In [0]:
#Train
train = np.insert(train, train.shape[1], 'distance', axis=1)
toDelete = []
for i in range(1,train.shape[0]):
  if((float)(train[i,2])<-90 or (float)(train[i,2])>90 or (float)(train[i,3])<-90 or (float)(train[i,3])>90 
     or (float)(train[i,4])<-90 or (float)(train[i,4])>90 or (float)(train[i,5])<-90 or (float)(train[i,5])>90):
    toDelete = np.append(toDelete,i)
  else:
    c1 = (train[i,2], train[i,3]) #coord. 1
    c2 = (train[i,4], train[i,5]) #coord. 2
    train[i,train.shape[1]-1] = geopy.distance.vincenty(c1, c2).km
train = np.delete(train,[toDelete], axis=0)
# In quello continuo NON bisogna aggiungere speed e negli altri due solo in uno ci deve essere
train = np.insert(train, train.shape[1], 'speed', axis=1)
for i in range(1,train.shape[0]):
  a=(float)(train[i,(train.shape[1]-2)])/((float)(train[i,6])/3600)
  train[i,(train.shape[1]-1)] = str(a)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  # This is added back by InteractiveShellApp.init_path()


Aggiungere due colonne che indicano il quartiere di partenza e di arrivo del taxi:

1: Manhattan, 2: Brooklyn, 3: Queens, 4: Bronx, 5: Staten Island, 6: Altro

In [0]:
zone[0,0] = "longitudine"
zone[0,1] = "latitudine"
zone = np.delete(zone,np.s_[2:8],axis=1)

for i in range(1,zone.shape[0]):
  if(zone[i,2] == 'Manhattan'):
    zone[i,2] = 1
  if(zone[i,2] == 'Brooklyn'):
    zone[i,2] = 2
  if(zone[i,2] == 'Queens'):
    zone[i,2] = 3
  if(zone[i,2] == 'Bronx'):
    zone[i,2] = 4
  if(zone[i,2] == 'Staten Island'):
    zone[i,2] = 5
  zone_lon_lat = re.sub(r'([^\-\s\w\.]|_)+', '', str(zone[i,0]))
  zone_lon_lat = zone_lon_lat.replace("POINT","").split()
  zone[i,0] = float(zone_lon_lat[0])
  zone[i,1] = float(zone_lon_lat[1])
  
#METODO CON CENTROIDE
train = np.insert(train, train.shape[1], 'start_zone', axis=1)
train = np.insert(train, train.shape[1], 'end_zone', axis=1)


def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

for i in range(1,train.shape[0]):
  counter_start = np.zeros(5)
  counter_end = np.zeros(5)
  for j in range(1,zone.shape[0]):
    center_point = [{'lat': zone[j,1], 'lng': zone[j,0]}]
    test_point_start = [{'lat': train[i,3], 'lng':  train[i,2]}]
    test_point_end = [{'lat': train[i,5], 'lng': train[i,4]}]

    lat_center = float(center_point[0]['lat'])
    lon_center = float(center_point[0]['lng'])
    lat_start = float(test_point_start[0]['lat'])
    lon_start = float(test_point_start[0]['lng'])
    lat_end = float(test_point_end[0]['lat'])
    lon_end = float(test_point_end[0]['lng'])

    radius = 3.00 # in kilometer - fare varie prove 2-4-6 bho?!

    dist_start = haversine(lon_center, lat_center, lon_start, lat_start)
    dist_end = haversine(lon_center, lat_center, lon_end, lat_end)
    if dist_start <= radius:
      # Inside the area
      counter_start[int(zone[j,2])-1] =  counter_start[int(zone[j,2])-1] + 1
    if dist_end <= radius:
      # Inside the area
      counter_end[int(zone[j,2])-1] =  counter_end[int(zone[j,2])-1] + 1 
  
  if(np.sum(counter_start) > 0):
    train[i,(train.shape[1]-2)] = counter_start.argmax(axis=0) + 1
  else:
    train[i,(train.shape[1]-2)] = 6
  if(np.sum(counter_end) > 0):
    train[i,(train.shape[1]-1)] = counter_end.argmax(axis=0) + 1
  else:
    train[i,(train.shape[1]-1)] = 6
    
#elimina righe con 6 - valore non classificato
print(train.shape)

toDelete = []
for i in range(1,train.shape[0]):
  if(train[i,train.shape[1]-1] == 6 or train[i,train.shape[1]-2] == 6 ):
    toDelete = np.append(toDelete,i)
train = np.delete(train, np.s_[toDelete], axis=0)

print("START ZONE ---------------------------")
print(pd.value_counts(train[1:,train.shape[1]-2]))
print("END ZONE -----------------------------")
print(pd.value_counts(train[1:,train.shape[1]-1]))

(1436567, 28)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


START ZONE ---------------------------
1    1315623
3      67559
2      26575
4       3800
5         63
dtype: int64
END ZONE -----------------------------
1    1259426
2      73560
3      58917
4      21349
5        368
dtype: int64


### 2.4) Analisi del Dataset e ulteriori modifiche

Analisi dei valori nulli

In [0]:
# Contare i valori nulli
nullvalues = np.zeros(train.shape[1])
for j in range(0,train.shape[1]): #for each column
  for i in range(1,train.shape[0]): #for each row
    if(train[i,j]=="" or train[i,j]==" " or train[i,j] is None):
      nullvalues[j]=nullvalues[j]+1
# Rimuovere le colonne con troppi valori nulli (>1/3) e, successivamente,
# le righe con valori nulli
toDelete = []
print('Null values for each column:')
for i in range(0,len(nullvalues)):
  print('Column:',train[0,i],', # null values:',nullvalues[i])
  if(nullvalues[i]>train.shape[0]/3):
    toDelete = np.append(toDelete, i)
train = np.delete(train, [toDelete], axis=1)
toDelete = []
for i in range(1,train.shape[0]):
  delete = 0
  for j in range(0,train.shape[1]):
    if(train[i,j]=="" or train[i,j]==" " or train[i,j] is None):
      delete = 1
  if(delete == 1):
    toDelete = np.append(toDelete,i)
train = np.delete(train,[toDelete], axis=0)

Null values for each column:
Column: hour , # null values: 0.0
Column: passenger_count , # null values: 0.0
Column: pickup_longitude , # null values: 0.0
Column: pickup_latitude , # null values: 0.0
Column: dropoff_longitude , # null values: 0.0
Column: dropoff_latitude , # null values: 0.0
Column: trip_duration , # null values: 0.0
Column: tempm , # null values: 1103.0
Column: dewptm , # null values: 1103.0
Column: hum , # null values: 1103.0
Column: wspdm , # null values: 109888.0
Column: wgustm , # null values: 1122763.0
Column: vism , # null values: 38153.0
Column: pressurem , # null values: 37669.0
Column: windchillm , # null values: 1015944.0
Column: heatindexm , # null values: 1352033.0
Column: precipm , # null values: 1295417.0
Column: conds , # null values: 0.0
Column: fog , # null values: 0.0
Column: rain , # null values: 0.0
Column: snow , # null values: 0.0
Column: dayweek , # null values: 0.0
Column: holidays , # null values: 0.0
Column: month , # null values: 0.0
Column: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


### 2.5) Discretizzazione di alcune features

Discretizzo raggruppando in classi i valori di velocità, tirp duration e distanza

In [0]:
# Convertire i valori a float
train[1:,:] = train[1:,:].astype(float)

train = np.insert(train, train.shape[1], 'speed_disc', axis=1)
train = np.insert(train, train.shape[1], 'trip_duration_disc', axis=1)
train = np.insert(train, train.shape[1], 'distance_disc', axis=1)

print(train[0])

tripDurationColumn = 6
tripDurationDiscColumn = 25

speedColumn = 21
speedDiscColumn = 24

distanceColumn = 20
distanceDiscColumn = 26

toDelete = []
for i in range(1,train.shape[0]):
  speed = train[i,(speedColumn)]
  if(speed > 0 and speed <= 6):
    train[i,speedDiscColumn] = 0
  elif(speed > 6 and speed <= 10):
    train[i,speedDiscColumn] = 1
  elif(speed > 10 and speed <= 15):
    train[i,speedDiscColumn] = 2
  elif(speed > 16 and speed <=50):
    train[i,speedDiscColumn] = 3
  else:
    toDelete = np.append(toDelete, i)
  

train = np.delete(train, [toDelete], axis=0) 
print('SPEED--------------',len(toDelete))
print(pd.value_counts(train[1:,speedDiscColumn]))


toDelete = []
for i in range(1,train.shape[0]):
  tripDuration = train[i,(tripDurationColumn)]
  if(tripDuration > 0 and tripDuration <= 300): # meno di 5 minuti
    train[i,tripDurationDiscColumn] = 0
  elif(tripDuration > 300 and tripDuration <= 600): # da 5 a 10 minuti
    train[i,tripDurationDiscColumn] = 1
  elif(tripDuration > 600 and tripDuration <= 1500): # da 10 a 25
    train[i,tripDurationDiscColumn] = 2
  elif(tripDuration > 1500 and tripDuration <= 2400):   # da 25 a 40
    train[i,tripDurationDiscColumn] = 3
  elif(tripDuration > 2400 and tripDuration <= 7200):   # da 40 a 2 ore
    train[i,tripDurationDiscColumn] = 4
  else:
    toDelete = np.append(toDelete, i)

train = np.delete(train, [toDelete], axis=0)   
print('TRIP DURATION--------------', len(toDelete))
print(pd.value_counts(train[1:,tripDurationDiscColumn]))


toDelete = []
for i in range(1,train.shape[0]):
  distance = train[i,(distanceColumn)]
  if(distance > 0 and distance <= 0.7):
    train[i,distanceDiscColumn] = 0
  elif(distance > 0.7 and distance <= 1.3): 
    train[i,distanceDiscColumn] = 1
  elif(distance > 1.3 and distance <= 2): 
    train[i,distanceDiscColumn] = 2
  elif(distance > 2 and distance <= 3):  
    train[i,distanceDiscColumn] = 3
  elif(distance > 3 and distance <= 15):                       
    train[i,distanceDiscColumn] = 4
  else:
    toDelete = np.append(toDelete, i)
    
print('DISTANCE--------------',len(toDelete))
train = np.delete(train, [toDelete], axis=0)    
print(pd.value_counts(train[1:,distanceDiscColumn]))

['hour' 'passenger_count' 'pickup_longitude' 'pickup_latitude'
 'dropoff_longitude' 'dropoff_latitude' 'trip_duration' 'tempm' 'dewptm'
 'hum' 'wspdm' 'vism' 'pressurem' 'conds' 'fog' 'rain' 'snow' 'dayweek'
 'holidays' 'month' 'distance' 'speed' 'start_zone' 'end_zone'
 'speed_disc' 'trip_duration_disc' 'distance_disc']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


SPEED-------------- 49895
0    345519
1    340409
2    318307
3    249118
dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:54: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


TRIP DURATION-------------- 1902
2    548066
1    375605
0    188482
3    108542
4     30756
dtype: int64
DISTANCE-------------- 24171


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future


1    305534
4    248523
2    243367
0    239882
3    189974
dtype: int64


## 3) Download del Dataset

Salvataggio su drive del dataset creato



In [0]:
df = pd.DataFrame(train[:,:])
df.to_csv('DatasetFinale.csv')
!cp DatasetFinale.csv drive/My\ Drive/AdvancedMachineLearning/Dataset/